In [1]:
import pickle

In [4]:
q_table = pickle.load(open('BlackJack-lecture-01.pkl', 'rb'))

In [5]:
q_table

{(15, 6, False): [-2.6672208262076094, -3.043758739061678],
 (13, 1, False): [-3.2309629568940674, -3.0881542317699515],
 (18, 9, False): [-2.6476304866365945, -3.229435079027964],
 (12, 9, False): [-3.1342295043498476, -3.0387880541371737],
 (7, 1, False): [-3.490556253673852, -2.88717858349435],
 (21, 3, False): [-1.6951251100181512, -3.594670784511627],
 (17, 3, False): [-2.6022455065683308, -3.197989320985093],
 (19, 10, False): [-2.56812231363041, -3.3594895380557155],
 (10, 10, False): [-3.048071689611145, -2.593064555892486],
 (17, 1, False): [-3.1729056509388416, -3.2807575366498956],
 (12, 10, False): [-3.1679752791129023, -2.980931345635177],
 (14, 10, False): [-3.2598736901861027, -3.049828842913976],
 (14, 5, False): [-2.72892089615044, -3.12601863833007],
 (12, 6, False): [-2.791325028275523, -2.807937552229109],
 (18, 1, False): [-3.024639740862042, -3.2751251002541544],
 (14, 3, False): [-2.7717172278427307, -3.025921645813729],
 (20, 1, False): [-2.4171653532460016, -3.

In [6]:
def policy(state, episode=0.2): 
    
    global Q_table
    
    if random.random() < episode:
        return np.random.choice(range(len(Q_table[state])))
    else:
        return np.argmax(Q_table[state])

In [42]:
import gymnasium as gym
env = gym.make('Blackjack-v1', natural=True, sab=True, render_mode='rgb_array')

In [43]:
player_sum = set()
dealer = set()

In [44]:
for i in range(100000):
    (p, d, t), _ = env.reset()
    player_sum.add(p)
    dealer.add(d)

In [46]:
dealer

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

In [16]:
r = env.np_random

In [22]:
deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]

In [28]:
r.choice(deck)

5

## 游戏规则：

1. 玩家与AI各领1组扑克；（需要花费1个筹码）
2. 玩家与AI轮流发言，可以采取"hit"或者"stick"两种策略
3. 若某玩家/AI发生爆炸，则该玩家直接算输，且退出该场次比赛，剩余的一位玩家直接获胜；
4. 若所有玩家全部“stick”，则此局游戏结束，离21点最接近的人获胜，若有多人相等，则为平均！

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from blackjack_pk import BlackJack2PeoplePK, Judge

In [20]:
import random

In [131]:
from blackjack_pk import is_bust

In [141]:
agent1, agent2 = BlackJack2PeoplePK("Minquan"), BlackJack2PeoplePK("robot")
judeger = Judge(agent1, agent2)

distributor = judeger.give_new_hand()

agent1.reset(next(distributor), next(distributor))
agent2.reset(next(distributor), next(distributor))

reward = 0 

action_name = {
    0: "STICK", 
    1: "HIT"
}

while True:
    # Agent-1
        
    action1 = random.choice(agent1.action_space)
    
    assert not agent1.is_bust()
    
    new_card = next(distributor) if action1 == 1 else None
    
    print(f"{agent1.name} {action_name[action1]}")
    player_1_cards, opponetn_actions, ace = agent1.step(action1, agent2, new_card)
    
    if agent1.is_bust(): 
        reward = -1 
        done = True
        
    # Agent-2

    action2 = random.choice(agent2.action_space)

    print(f"{agent2.name} {action_name[action2]}")
    
    new_card = next(distributor) if action2 == 1 else None
    
    assert not agent2.is_bust()
    
    player_2_cards, opponetn_actions, ace = agent2.step(action2, agent1, new_card)
    
    if agent2.is_bust():  # agent2 bust, agent1 win
        reward = 1
        done = True
    
    # one-round-is-end
    
    if not done and (action1 == action2 == 0): 
        done = True
        if agent1.card_sum > agent2.card_sum: reward = 1
        elif agent1.card_sum == agent2.card_sum: reward = 0.5
        else:
            reward = -1  # agent1 loss
    else:
        done = False

    p1, p2 = judeger.render()

    print(p1)
    print(p2)

    if terimiated_1 or terimiated_2:
        print('result: ')
        print(terimiated_1, terimiated_2)
        if reward == 1:  print(f'Winer: {agent1.name}')
        elif reward == -1: print(f'Winer: {agent2.name}')
        else:
            print('平局')
        print('Done')
        break

Minquan STICK
robot HIT
Minquan: 1-card, : Spades-A
Minquan: 2-card, : Diamonds-7
robot: 1-card, : [b'gAAAAABjxigS_O8u87_qv7Z1KoO1JGkZg7Re56IluO8HmcQ4X3tJKWm44ojjt7QMpv0h8yN-s11eYBqR2dlcUtEdDW2wyhJ3dg==', b'gAAAAABjxigSajygcBgjAnnjH_eInG1xBlAA_SlnMyAdoELjKnjMQ1rM6W_6CaDPLTPAww3u1i2C5yuPG3HVKj8_r_M8gIC2nQ==', b'gAAAAABjxigSCIwXIjYeNzMLsa1XGCNOhpzLGmVJYvnWLqQDZhtRYKxt8LrDb-SihvVf-Gjs6UkHXfzneBMeabbvHFgchnJixQ==']
robot: 2-card, : [b'gAAAAABjxigSsaaHTZnm2ppjLyZ2yjfWv6MchQw-P5Jv1MDwxbjmDYW8AnQi90d_NqceCe2TbdY84701uqVr4nqDnGKu5S0ahg==', b'gAAAAABjxigSN7OoNwzDY6VIP-8ZwMzmiF4TNRNCtIyhQaaGyR-twIrjxkiPRBXxkS-KU7XrfY_bZdPTEv905rMfVsD0lKbc9w==', b'gAAAAABjxigShrkHidYyl189vFGVRjnbXtEy8l9iy7cWR65SaH9KcRH23kK1qkQT5G61iHelGaj-jx8NIzkbEU3HtB3dw5pTvw==']
robot: 3-card, : [b'gAAAAABjxigSBxXZMLwuGl1A1JFN6VkoOJ4AWcDewGrGzZ615HuhjY9yeye08PUOPr7c_9FkG82eR-j4H6-9ZSgulHS5DsF6cg==', b'gAAAAABjxigS1jjSk9Qqq1lPTYK5nPBgzgRUBdIMS8v2Pw8NxCzidSrLrNTGVsoCwdwr7zcRStry49tHwNYp1d0RMZ0wb6uHSA==', b'gAAAAABjxigSjtHMWRJBcK

In [142]:
judeger.decode()

['Diamonds-9', 'Diamonds-8', 'Hearts-3']

In [149]:
agent2.card_sum

20